<a href="https://colab.research.google.com/github/ritwiks9635/NLP_Project_File/blob/main/Next_Word_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Next Word Prediction**🇳🇺🇧🇼🇨🇼🇽🇰🇹🇰🇸🇷🇩🇯🇩🇿🇫🇯🇲🇬🇲🇬🇲🇷🇲🇷

In [ ]:
! unzip /content/https:/www.kaggle.com/datasets/johnybhiduri/us-crime-data/us-crime-data.zip

Archive:  /content/https:/www.kaggle.com/datasets/johnybhiduri/us-crime-data/us-crime-data.zip
  inflating: US_Crime_Data.csv       


In [ ]:
import re
import string
import numpy as np
import pandas as pd
import tensorflow as tf

In [ ]:
data = pd.read_csv("US_Crime_Data.csv")
data.head()

,Date,Title,Organization,City,State,URL,Keyword,Summary
0,2/13/17 9:01,Pizza Hut driver who killed co-worker with sho...,LGBTQ Nation,Phoenix,AZ,http://www.lgbtqnation.com/2017/02/man-accused...,NaN,NaN
1,2/13/17 9:37,Residents of NJ township receiving KKK promoti...,New York Daily News,NaN,NaN,http://www.nydailynews.com/news/national/resid...,NaN,NaN
2,2/13/17 15:37,House OKs bill to expand Kentucky's hate crime...,FOX19,Cincinnati,OH,http://www.fox19.com/story/34494432/house-oks-...,NaN,NaN
3,2/13/17 16:56,"Amid Protests, 'Blue Lives Matter' Bill Passes...",89.3 WFPL,Louisville,KY,http://wfpl.org/amid-protests-blue-lives-matte...,NaN,NaN
4,2/13/17 18:37,Lafourche inmates charged with hate crimes in ...,Daily Comet,Thibodaux,LA,http://www.dailycomet.com/news/20170213/lafour...,NaN,NaN


In [ ]:
data.shape

(6783, 8)

In [ ]:
data.isnull().sum()

Date               0
Title              1
Organization       0
City            1167
State           1245
URL                0
Keyword         1176
Summary         2256
dtype: int64

In [ ]:
df = data[["Title"]]
df = df.dropna().reset_index(drop = True)

In [ ]:
df.head()

,Title
0,Pizza Hut driver who killed co-worker with sho...
1,Residents of NJ township receiving KKK promoti...
2,House OKs bill to expand Kentucky's hate crime...
3,"Amid Protests, 'Blue Lives Matter' Bill Passes..."
4,Lafourche inmates charged with hate crimes in ...


In [ ]:
def clean_data(df):
  df["Title"] = df["Title"].apply(lambda x : x.lower())
  text = df["Title"].str.replace('[{}]'.format(string.punctuation), ' ')
  return text

In [ ]:
text = clean_data(df)

<ipython-input-19-edd9664593aa>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  text = df["Title"].str.replace('[{}]'.format(string.punctuation), ' ')


In [ ]:
text

0       pizza hut driver who killed co worker with sho...
1       residents of nj township receiving kkk promoti...
2       house oks bill to expand kentucky s hate crime...
3       amid protests   blue lives matter  bill passes...
4       lafourche inmates charged with hate crimes in ...
                              ...                        
6777    police  rash of vandalism on margate menorahs ...
6778    ex student accused of smearing used tampon on ...
6779    queensbury woman charged with hate crime again...
6780    hate crime  hartford student accused of smeari...
6781    police make arrest following alleged hate crim...
Name: Title, Length: 6782, dtype: object

In [ ]:
text.shape

(6782,)

In [ ]:
len(set(text))

6567

In [ ]:
token = tf.keras.preprocessing.text.Tokenizer()
token.fit_on_texts(text)
word_index = token.word_index
sequence = token.texts_to_sequences(text)

In [ ]:
word_index

In [ ]:
sequence[:4]

[[3878, 3879, 437, 54, 94, 919, 703, 13, 2203, 109, 1388, 2771],
 [852, 4, 398, 1221, 2204, 438, 3880, 704],
 [262, 1845, 82, 5, 617, 618, 12, 1, 6, 37],
 [752, 920, 497, 286, 316, 82, 498, 618, 262]]

In [ ]:
x = []
y = []
total_words_drop = 0
for i in sequence:
    if len(i) > 1:
        for j in range(1, len(i)):
            x.append(i[:j])
            y.append(i[j])

    else :
        total_words_drop +=1
print('Total Words Dropped : {}'.format(total_words_drop))

Total Words Dropped : 11


In [ ]:
x[:10]

[[3878],
 [3878, 3879],
 [3878, 3879, 437],
 [3878, 3879, 437, 54],
 [3878, 3879, 437, 54, 94],
 [3878, 3879, 437, 54, 94, 919],
 [3878, 3879, 437, 54, 94, 919, 703],
 [3878, 3879, 437, 54, 94, 919, 703, 13],
 [3878, 3879, 437, 54, 94, 919, 703, 13, 2203],
 [3878, 3879, 437, 54, 94, 919, 703, 13, 2203, 109]]

In [ ]:
y[:10]

[3879, 437, 54, 94, 919, 703, 13, 2203, 109, 1388]

In [ ]:
max(len(i) for i in sequence)

52

In [ ]:
x = tf.keras.preprocessing.sequence.pad_sequences(x)

In [ ]:
x.shape

(66996, 51)

In [ ]:
y = tf.keras.utils.to_categorical(y)

In [ ]:
y[:3]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
y.shape

(66996, 7116)

In [ ]:
vocab_size = len(word_index) + 1
vocab_size

7116

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, 49),
    tf.keras.layers.LSTM(100, return_sequences = True),
    tf.keras.layers.LSTM(100),
    tf.keras.layers.Dense(100, activation = "relu"),
    tf.keras.layers.Dense(vocab_size, activation = "softmax")
    ])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, None, 49)          348684    
                                                                 
 lstm (LSTM)                 (None, None, 100)         60000     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7116)              718716    
                                                                 
Total params: 1,217,900
Trainable params: 1,217,900
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics=["accuracy"])

In [ ]:
e_s = tf.keras.callbacks.EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True)

In [ ]:
model.fit(x, y, epochs = 100, batch_size = 256, callbacks = e_s)

Epoch 1/100
262/262 [==============================] - ETA: 0s - loss: 6.9014 - accuracy: 0.0639

262/262 [==============================] - 40s 116ms/step - loss: 6.9014 - accuracy: 0.0639
Epoch 2/100
262/262 [==============================] - ETA: 0s - loss: 6.3981 - accuracy: 0.0982

262/262 [==============================] - 9s 36ms/step - loss: 6.3981 - accuracy: 0.0982
Epoch 3/100
262/262 [==============================] - ETA: 0s - loss: 6.2346 - accuracy: 0.1137

262/262 [==============================] - 7s 27ms/step - loss: 6.2346 - accuracy: 0.1137
Epoch 4/100
262/262 [==============================] - ETA: 0s - loss: 6.0451 - accuracy: 0.1249

262/262 [==============================] - 6s 24ms/step - loss: 6.0451 - accuracy: 0.1249
Epoch 5/100
262/262 [==============================] - ETA: 0s - loss: 5.8685 - accuracy: 0.1296

...

262/262 [==============================] - 5s 19ms/step - loss: 1.7670 - accuracy: 0.5972
Epoch 97/100
261/262 [============================>.] - ETA: 0s - loss: 1.7556 - accuracy: 0.5994

262/262 [==============================] - 4s 16ms/step - loss: 1.7556 - accuracy: 0.5995
Epoch 98/100
260/262 [============================>.] - ETA: 0s - loss: 1.7473 - accuracy: 0.6016

262/262 [==============================] - 5s 17ms/step - loss: 1.7473 - accuracy: 0.6016
Epoch 99/100
259/262 [============================>.] - ETA: 0s - loss: 1.7357 - accuracy: 0.6015

262/262 [==============================] - 5s 19ms/step - loss: 1.7367 - accuracy: 0.6014
Epoch 100/100
261/262 [============================>.] - ETA: 0s - loss: 1.7236 - accuracy: 0.6053

262/262 [==============================] - 4s 16ms/step - loss: 1.7233 - accuracy: 0.6053


In [ ]:
vocab_keys = np.array(list(word_index.keys()))

In [ ]:
def prediction_text(text, n_words):
  for i in range(n_words):
    test_sequence = token.texts_to_sequences([text])
    text_padding  = tf.keras.preprocessing.sequence.pad_sequences(test_sequence, maxlen = 49)
    prediction = np.squeeze(np.argmax(model.predict(text_padding), axis = -1))
    prediction = str(vocab_keys[prediction - 1])
    text += " " + prediction
  return text

In [ ]:
prediction_text('california',5)

1/1 [==============================] - 0s 20ms/step


'california hate crimes rise in california'

In [ ]:
prediction_text('new york',8)

1/1 [==============================] - 0s 21ms/step


'new york police investigating hate crime at south orange county'

In [ ]:
prediction_text('india',8)

1/1 [==============================] - 0s 18ms/step


'india man who hoaxed hillary latina lynching on car'